In [20]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss

In [21]:
df = pd.read_csv("Social_Network_Ads.csv")
print(df.info())
print(df.describe())

# Imprimimos los valores unicos de cada columna para analizar su contenido
for col in df.columns:
    print(f"{col}: {df[col].unique()}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User ID          400 non-null    int64 
 1   Gender           400 non-null    object
 2   Age              400 non-null    int64 
 3   EstimatedSalary  400 non-null    int64 
 4   Purchased        400 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 15.8+ KB
None
            User ID         Age  EstimatedSalary   Purchased
count  4.000000e+02  400.000000       400.000000  400.000000
mean   1.569154e+07   37.655000     69742.500000    0.357500
std    7.165832e+04   10.482877     34096.960282    0.479864
min    1.556669e+07   18.000000     15000.000000    0.000000
25%    1.562676e+07   29.750000     43000.000000    0.000000
50%    1.569434e+07   37.000000     70000.000000    0.000000
75%    1.575036e+07   46.000000     88000.000000    1.000000
max    1.581524e+07   60.000000    15

In [22]:
# Eliminamos la columna User ID ya que no aporta nada al modelo
df = df.drop(["User ID"], axis=1)

# Encodeamos las columnas con valores categóricos
df["Gender"] = pd.get_dummies(df["Gender"], drop_first=True)

# Eligimos la variable a predecir y las predictoras
X = df.drop(["Purchased"], axis=1)
y = df["Purchased"]

# Prepocesamos las varibales numéricas mediante el StandarScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --------------------
# MÉTODOS DE BALANCEO DE CLASES
# --------------------
# 1. Sobremuestreo aleatorio
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# 2. SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# 3. Submuestreo aleatorio
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# 4. NearMiss (submuestreo basado en distancia)
nearmiss = NearMiss()
X_train_nm, y_train_nm = nearmiss.fit_resample(X_train, y_train)

# --------------------
# EVALUACIÓN DE MÉTODOS DE BALANCEO
# --------------------
resultados_balanceo = {}

def evaluar_balanceo(X_train_resampled, y_train_resampled, metodo):
    # Creación del modelo de RL
    model = LogisticRegression(C=0.1, solver='liblinear', max_iter=500)
    # Entreno el modelo con los datos de cada modelo
    model.fit(X_train_resampled, y_train_resampled)
    # Realizo predicciones con el modelo
    y_pred = model.predict(X_test)
    # Calculo la precisión del modelo sacando las métricas
    f1 = f1_score(y_test, y_pred) * 100
    auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1]) * 100
    resultados_balanceo[metodo] = (f1, auc)
    # Imprimimos los resultados
    print(f"\nResultados con {metodo}:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
    print(f"Precision: {precision_score(y_test, y_pred) * 100:.2f}%")
    print(f"Recall: {recall_score(y_test, y_pred) * 100:.2f}%")
    print(f"F1-Score: {f1:.2f}%")
    print(f"AUC-ROC: {auc:.2f}%")

evaluar_balanceo(X_train_ros, y_train_ros, "Sobremuestreo Aleatorio")
evaluar_balanceo(X_train_smote, y_train_smote, "SMOTE")
evaluar_balanceo(X_train_rus, y_train_rus, "Submuestreo Aleatorio")
evaluar_balanceo(X_train_nm, y_train_nm, "NearMiss")

# Seleccionar el mejor método de balanceo
mejor_metodo = max(resultados_balanceo, key=lambda k: resultados_balanceo[k])
print(f"\nEl mejor método de balanceo es: {mejor_metodo} con F1-Score: {resultados_balanceo[mejor_metodo][0]:.2f}% y AUC-ROC: {resultados_balanceo[mejor_metodo][1]:.2f}%")

# Obtener los datos balanceados del mejor método encontrado
if mejor_metodo == "Sobremuestreo Aleatorio":
    X_train_resampled, y_train_resampled = X_train_ros, y_train_ros
elif mejor_metodo == "SMOTE":
    X_train_resampled, y_train_resampled = X_train_smote, y_train_smote
elif mejor_metodo == "Submuestreo Aleatorio":
    X_train_resampled, y_train_resampled = X_train_rus, y_train_rus
elif mejor_metodo == "NearMiss":
    X_train_resampled, y_train_resampled = X_train_nm, y_train_nm

# --------------------
# MÉTODO 1: Ajuste manual de hiperparámetros
# --------------------
manual_model = LogisticRegression(C=0.1, solver='liblinear', max_iter=500)  # Ejemplo de hiperparámetros ajustados manualmente
manual_model.fit(X_train_resampled, y_train_resampled)  # Entrenar modelo

# Predicciones
y_pred_manual = manual_model.predict(X_test)

# Evaluación del modelo ajustado manualmente
print("\nResultados con ajuste manual de hiperparámetros:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"Precision: {precision_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"Recall: {recall_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"F1-Score: {f1_score(y_test, y_pred_manual) * 100:.2f}%")
print(f"AUC-ROC: {roc_auc_score(y_test, manual_model.predict_proba(X_test)[:,1]) * 100:.2f}%")

# --------------------
# MÉTODO 2: Ajuste automático con GridSearchCV
# --------------------
parametros = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100],  # Diferentes valores de regularización
    "solver": ["liblinear", "lbfgs"]  # Diferentes algoritmos de optimización
}

# Configurar GridSearchCV
grid_search = GridSearchCV(LogisticRegression(max_iter=500), parametros, cv=5, scoring="accuracy", n_jobs=-1)

grid_search.fit(X_train_resampled, y_train_resampled)  # Entrenar búsqueda de hiperparámetros

# Mejor modelo encontrado
best_model = grid_search.best_estimator_

# Predicciones del mejor modelo
y_pred_best = best_model.predict(X_test)

# Evaluación del mejor modelo
print("\nResultados con ajuste automático de hiperparámetros:")
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor Accuracy en validación cruzada: {grid_search.best_score_ * 100:.2f}%")
print(f"Accuracy: {accuracy_score(y_test, y_pred_best) * 100:.2f}%")
print(f"Precision: {precision_score(y_test, y_pred_best) * 100:.2f}%")
print(f"Recall: {recall_score(y_test, y_pred_best) * 100:.2f}%")
print(f"F1-Score: {f1_score(y_test, y_pred_best) * 100:.2f}%")
print(f"AUC-ROC: {roc_auc_score(y_test, best_model.predict_proba(X_test)[:,1]) * 100:.2f}%")


# --------------------
# ENTRENAMIENTO FINAL Y EVALUACIÓN
# --------------------

# Entrenar el mejor modelo con los datos balanceados
best_model.fit(X_train_resampled, y_train_resampled)

# Realizar predicciones finales con el mejor modelo y el mejor balanceo
y_pred_final = best_model.predict(X_test)

# Crear DataFrame de resultados
resultados = pd.DataFrame({
    "Real": y_test.values,
    "Predicción": y_pred_final
})

# Mapear valores 0 y 1 a etiquetas comprensibles
resultados["Real"] = resultados["Real"].map({0: "No Comprado", 1: "Comprado"})
resultados["Predicción"] = resultados["Predicción"].map({0: "No Comprado", 1: "Comprado"})

# Mostrar las primeras filas de la tabla
print(resultados.head(10))


Resultados con Sobremuestreo Aleatorio:
Accuracy: 88.75%
Precision: 82.76%
Recall: 85.71%
F1-Score: 84.21%
AUC-ROC: 97.05%

Resultados con SMOTE:
Accuracy: 88.75%
Precision: 82.76%
Recall: 85.71%
F1-Score: 84.21%
AUC-ROC: 97.12%

Resultados con Submuestreo Aleatorio:
Accuracy: 87.50%
Precision: 80.00%
Recall: 85.71%
F1-Score: 82.76%
AUC-ROC: 97.12%

Resultados con NearMiss:
Accuracy: 88.75%
Precision: 91.30%
Recall: 75.00%
F1-Score: 82.35%
AUC-ROC: 96.77%

El mejor método de balanceo es: SMOTE con F1-Score: 84.21% y AUC-ROC: 97.12%

Resultados con ajuste manual de hiperparámetros:
Accuracy: 88.75%
Precision: 82.76%
Recall: 85.71%
F1-Score: 84.21%
AUC-ROC: 97.12%

Resultados con ajuste automático de hiperparámetros:
Mejores parámetros: {'C': 1, 'solver': 'liblinear'}
Mejor Accuracy en validación cruzada: 84.88%
Accuracy: 88.75%
Precision: 82.76%
Recall: 85.71%
F1-Score: 84.21%
AUC-ROC: 97.25%
          Real   Predicción
0  No Comprado  No Comprado
1     Comprado     Comprado
2  No Comp

In [29]:
# --------------------
# PRUEBA Y EJECUCIÓN DEL MODELO
# --------------------


# Buscamos un registro de un paciente con cancer para pasar los mismos datos y probar que el modelo predice correctamente
print(df[df["Purchased"] == 1].index) # Seleccionamos la 37
print(df.loc[7])

# Mapeo de características a nombres en castellano (opcional)
nombres_caracteristicas = {
    'Gender': 'Genero',
    'Age': 'Edad',
    'EstimatedSalary': 'Salario estimado',
}

# Creamos la función para ejecutar el modelo con nuevos datos
def solicitar_datos_y_predecir(modelo, feature_names, scaler):
    print("\nIngrese los valores para realizar una predicción:")
    valores = []
    
    for feature in feature_names:
        nombre_caracteristica = nombres_caracteristicas.get(feature, feature)  
        while True:
            try:
                valor = float(input(f"{nombre_caracteristica}: "))
                valores.append(valor)
                break
            except ValueError:
                print("Entrada inválida. Ingrese un número válido.")
    
    # Convertir la entrada en un array de numpy y escalarlo con el scaler ya entrenado
    datos_nuevos = np.array(valores).reshape(1, -1)
    datos_nuevos_escalados = scaler.transform(datos_nuevos)
    
    # Realizar predicción
    prediccion = modelo.predict(datos_nuevos_escalados)[0]
    probabilidad = modelo.predict_proba(datos_nuevos_escalados)[0][prediccion] * 100
    
    # Mostrar resultado
    resultado = "Comprado" if prediccion == 1 else "No comprado"
    print(f"\nPredicción: {resultado} ({probabilidad:.2f}% de confianza)\n")

# Llamar a la función con el scaler ya ajustado
solicitar_datos_y_predecir(best_model, df.drop(columns=["Purchased"]).columns, scaler)

Index([  7,  16,  17,  18,  19,  20,  21,  22,  23,  24,
       ...
       388, 389, 390, 391, 392, 393, 395, 396, 397, 399],
      dtype='int64', length=143)
Gender              False
Age                    32
EstimatedSalary    150000
Purchased               1
Name: 7, dtype: object

Ingrese los valores para realizar una predicción:

Predicción: No comprado (91.29% de confianza)



c:\Users\SergioSF\Desktop\Programacion de Modelos de IA\Ejercicios y apuntes\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
